<a href="https://colab.research.google.com/github/ruoxipan/sentiment-analysis/blob/master/sml__tf_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import nltk
import string
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import roc_auc_score, roc_curve
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing import sequence 
from sklearn.ensemble import RandomForestClassifier
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.sequence import pad_sequences
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import LinearSVC

Using TensorFlow backend.


In [2]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

# Importing data from drive

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
train_df = pd.read_csv('/gdrive/My Drive/IMDb_review_dataset/train_data.csv')
test_df = pd.read_csv('/gdrive/My Drive/IMDb_review_dataset/test_data.csv')

In [5]:
display(train_df.head())
display(test_df.head())

,File_index,Reviews,Rating,Label
0,0,Bromwell High is a cartoon comedy. It ran at t...,9,1
1,10000,Homelessness (or Houselessness as George Carli...,8,1
2,10001,Brilliant over-acting by Lesley Ann Warren. Be...,10,1
3,10002,This is easily the most underrated film inn th...,7,1
4,10003,This is not the typical Mel Brooks film. It wa...,8,1


,File_index,Reviews,Rating,Label
0,0,I went and saw this movie last night after bei...,10,1
1,10000,Actor turned director Bill Paxton follows up h...,7,1
2,10001,As a recreational golfer with some knowledge o...,9,1
3,10002,"I saw this film in a sneak preview, and it is ...",8,1
4,10003,Bill Paxton has taken the true story of the 19...,8,1


In [0]:
train_X = train_df['Reviews'].tolist()
test_X = test_df['Reviews'].tolist()
train_labels = train_df['Label']
test_labels = test_df['Label']

# Accuracy without preprocessing data

In [0]:
wp_vectorizer = CountVectorizer(max_features=10000)
wp_cv_train = wp_vectorizer.fit_transform(train_X).toarray()
wp_cv_test = wp_vectorizer.fit_transform(test_X).toarray()

## Logistic regression

In [8]:
wp_cv_log = LogisticRegression()
wp_cv_log.fit(wp_cv_train, train_labels)
wp_cv_log_pred = wp_cv_log.predict(wp_cv_test)
print(accuracy_score(wp_cv_log_pred, test_labels))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.52152


## Linear SVC

In [9]:
wp_cv_svc = LinearSVC()
wp_cv_svc.fit(wp_cv_train, train_labels)
wp_cv_svc_pred = wp_cv_svc.predict(wp_cv_test)
print(accuracy_score(wp_cv_svc_pred, test_labels))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.52576


# DATA PREPROCESSING

## Removing html tags

In [0]:
#remove html tags
def strip_html_tag(sample):
    cleanr = re.compile(r"\<[^\>]*\>|\\")
    return re.sub(cleanr, '', sample)

In [0]:
train_X = list(map(strip_html_tag, train_X))
test_X = list(map(strip_html_tag, test_X))

## Converting all text to lowercase

In [0]:
#Converting string to lower case
def string_lower(sample):
    sample.replace('.',' . ')
    return sample.lower()

In [0]:
train_X = list(map(string_lower, train_X))
test_X = list(map(string_lower, test_X))

## Tokenize all texts

In [0]:
# Tokenize
def tokenize_text(sample):
    tokenizer = TweetTokenizer()
    return tokenizer.tokenize(sample)

In [0]:
train_X = list(map(tokenize_text,train_X))
test_X = list(map(tokenize_text,test_X))

## Remove stop words

In [0]:
# Remove stop words
def stopword_remover(sample):
    stopwords_list = set(stopwords.words('english'))
    sample_wostopword = []
    for i in sample:
        if i not in stopwords_list:
            sample_wostopword.append(i) 
    return sample_wostopword

In [0]:
train_X = list(map(stopword_remover,train_X))
test_X = list(map(stopword_remover,test_X))

## Remove punctuations

In [0]:
# Remove punctuations
def punctuation_remover(sample):
    sample_wostopword_wopunc = []
    for i in sample:
        if i not in string.punctuation:
            sample_wostopword_wopunc.append(i)
    return sample_wostopword_wopunc

In [0]:
train_X = list(map(punctuation_remover,train_X))
test_X = list(map(punctuation_remover,test_X))

## Adding parts of speech to each word

In [0]:
# Parts of speech
def text_pos_tag(sample):
    return nltk.pos_tag(sample)

In [0]:
train_X = list(map(text_pos_tag,train_X))
test_X = list(map(text_pos_tag,test_X))

## Lemmatize tokens

In [0]:
def lemmatize(token):
    word, tag = token
    
    lemmatizer = WordNetLemmatizer()
    
    if word not in list(".,;'\"-"):
        return lemmatizer.lemmatize(word, get_wordnet_pos(tag))
    
    return word

In [0]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [0]:
# Lemmatize all reviews
def lemmatized_text(sample):
    return list(map(lemmatize,sample))

In [0]:
train_X = list(map(lemmatized_text,train_X))
test_X = list(map(lemmatized_text,test_X))

## Cleaned text

In [0]:
# Combine token to form sentences as countvactorizer need sentence as input
train_X = list(map(lambda x: " ".join(x), train_X))
test_X = list(map(lambda x: " ".join(x), test_X))

# Convert text to document term matrix

## tfidf

In [0]:
# keras text to matrix
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words= 10000)
tokenizer.fit_on_texts(train_X)

In [0]:
x_train = tokenizer.texts_to_matrix(train_X, mode='tfidf')
x_test = tokenizer.texts_to_matrix(test_X, mode='tfidf')

In [29]:
print(x_train.shape)
print(tokenizer.word_index)
print(x_test.shape)

(25000, 10000)
{'movie': 1, 'film': 2, 'one': 3, 'make': 4, 'like': 5, 'see': 6, 'get': 7, 'good': 8, 'time': 9, 'well': 10, 'go': 11, 'character': 12, 'watch': 13, 'story': 14, 'even': 15, 'bad': 16, 'would': 17, 'really': 18, 'think': 19, 'show': 20, 'scene': 21, 'great': 22, 'much': 23, 'look': 24, 'say': 25, 'end': 26, 'people': 27, 'also': 28, 'first': 29, 'know': 30, 'give': 31, 'way': 32, 'take': 33, 'love': 34, 'come': 35, 'play': 36, 'thing': 37, 'act': 38, 'find': 39, 'could': 40, 'life': 41, 'seem': 42, 'the': 43, 'work': 44, 'two': 45, 'plot': 46, 'actor': 47, 'year': 48, 'many': 49, 'want': 50, 'never': 51, 'little': 52, 'best': 53, 'try': 54, 'ever': 55, 'man': 56, 'still': 57, 'part': 58, 'old': 59, 'back': 60, 'something': 61, 'director': 62, "i'm": 63, 'use': 64, 'real': 65, 'funny': 66, 'lot': 67, 'performance': 68, 'though': 69, 'star': 70, 'feel': 71, 'cast': 72, 'new': 73, 'woman': 74, '10': 75, 'another': 76, 'nothing': 77, 'big': 78, 'actually': 79, 'u': 80, 'poi

In [32]:
print(x_train)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.67593516 ... 0.         0.         0.        ]
 [0.         0.93511583 0.         ... 0.         0.         0.        ]
 ...
 [0.         2.44012669 0.9898343  ... 0.         0.         0.        ]
 [0.         1.58328873 2.91596292 ... 0.         0.         0.        ]
 [0.         0.         1.67593516 ... 0.         0.         0.        ]]


## frequency

In [0]:
tokenizer_f = tf.keras.preprocessing.text.Tokenizer(num_words= 6000)
tokenizer_f.fit_on_texts(train_X)

In [0]:
x_train_f = tokenizer_f.texts_to_matrix(train_X, mode = "count")
x_test_f = tokenizer_f.texts_to_matrix(test_X, mode = "count")

In [0]:
print(x_train_f.shape)
print(tokenizer_f.word_index)
print(x_test_f.shape)

(25000, 6000)
{'movie': 1, 'film': 2, 'one': 3, 'make': 4, 'like': 5, 'see': 6, 'get': 7, 'good': 8, 'time': 9, 'well': 10, 'go': 11, 'character': 12, 'watch': 13, 'story': 14, 'even': 15, 'bad': 16, 'would': 17, 'really': 18, 'think': 19, 'show': 20, 'scene': 21, 'great': 22, 'much': 23, 'look': 24, 'say': 25, 'end': 26, 'people': 27, 'also': 28, 'first': 29, 'know': 30, 'give': 31, 'way': 32, 'take': 33, 'love': 34, 'come': 35, 'play': 36, 'thing': 37, 'act': 38, 'find': 39, 'could': 40, 'life': 41, 'seem': 42, 'the': 43, 'work': 44, 'two': 45, 'plot': 46, 'actor': 47, 'year': 48, 'many': 49, 'want': 50, 'never': 51, 'little': 52, 'best': 53, 'try': 54, 'ever': 55, 'man': 56, 'still': 57, 'part': 58, 'old': 59, 'back': 60, 'something': 61, 'director': 62, "i'm": 63, 'use': 64, 'real': 65, 'funny': 66, 'lot': 67, 'performance': 68, 'though': 69, 'star': 70, 'feel': 71, 'cast': 72, 'new': 73, 'woman': 74, '10': 75, 'another': 76, 'nothing': 77, 'big': 78, 'actually': 79, 'u': 80, 'poin

In [0]:
print(tokenizer.word_index==tokenizer_f.word_index)

True


## Count Vectorizer

In [0]:
vectorizer = CountVectorizer(max_features=10000)
cv_train = vectorizer.fit_transform(train_X).toarray()
cv_test = vectorizer.fit_transform(test_X).toarray()

In [0]:
print(cv_test == cv_train)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: SparseEfficiencyWarning: Comparing sparse matrices using == is inefficient, try using != instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [0]:
vectorized_df

,00,000,01,10,100,1000,101,11,11th,12,13,13th,14,15,150,16,17,18,180,18th,19,1920,1920s,1922,1928,1930,1930s,1931,1932,1933,1934,1935,1936,1937,1938,1939,1940,1940s,1941,1942,...,yo,yoda,yokai,york,you,young,youngster,your,yourself,youth,youthful,youtube,yr,yugoslavia,yup,yuppie,yuzna,yvonne,zabriskie,zane,zany,zatoichi,zealand,zelah,zelda,zenia,zero,zeta,zhang,zizek,zodiac,zoe,zombi,zombie,zombies,zone,zoo,zoom,zorro,zu
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24998,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0


In [0]:
x_train_n = tf.keras.utils.normalize(x_train, axis=1)
x_test_n = tf.keras.utils.normalize(x_test, axis=1)

In [0]:
def sigmoid(X,theta):
    x = (1/(1 + np.exp(-1*np.dot(X,theta))))
    return x

In [0]:
def cos_fun(predictions):
    J = -1*(np.dot(test_y_log.T,np.log(predictions + 0.00000001)) + 
            np.dot((1-test_y_log.T),np.log(1 - predictions + 0.00000001)))
    return J

In [0]:
def logis_gradient_descent(X,y,alpha,iterations):
    theta = np.random.random((10000, 1))
    for i in range(iterations):
        theta = theta - alpha*(np.dot(X.T,(sigmoid(X,theta) - y)))
    pred = np.where(sigmoid(x_test_n,theta)>=0.5,1,0)
    conf_mat = confusion_matrix(test_labels, pred)
    TN = conf_mat[0][0]
    FP = conf_mat[0][1]
    FN = conf_mat[1][0]
    TP = conf_mat[1][1]
    Accuracy = (TP+TN)/(TP+FP+TN+FN)
    Precision = TP/(TP+FP)
    Recall = TP/(TP+FN)
    f1_score = (2*Precision*Recall)/(Precision + Recall)
    CEL = cos_fun(sigmoid(test_X_std,theta))
    result = pd.Series([alpha,iterations,Accuracy, f1_score, CEL], 
                       index=['Alpha','Iterations','Accuracy','F1 Score','Cross Entropy Loss'])
    return result

In [0]:
alphas =  [0.0001]
#iterations = [1,3,5,8,10,15,25,50,100]
iterations = [1]
df_1d = pd.DataFrame(columns=['Alpha','Iterations','Accuracy','F1 Score', 'Cross Entropy Loss'])
for i in alphas:
    for j in iterations:
        df_1d = df_1d.append(logis_gradient_descent(x_train_n,train_labels,i,j), ignore_index = True)
display(df_1d)

# Classifiers

## Model 1 - Multinomial Naive Bayes

In [39]:
clf = MultinomialNB()
clf.fit(x_train, train_labels)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [40]:
y_pred = clf.predict(x_test)
print(accuracy_score(test_labels, y_pred))

0.81796


## Evaluation Metrics

In [0]:
def conf_matrix(y_test,pred):
    conf_mat = confusion_matrix(y_test, pred)
    print(conf_mat)
    TN = conf_mat[0][0]
    FP = conf_mat[0][1]
    FN = conf_mat[1][0]
    TP = conf_mat[1][1]
    Accuracy = (TP+TN)/(TP+FP+TN+FN)
    Precision = TP/(TP+FP)
    Recall = TP/(TP+FN)
    f1_score = (2*Precision*Recall)/(Precision + Recall)
    result = pd.DataFrame({'Accuracy':[Accuracy], 'Precision':[Precision], 'Recall':[Recall], 'f1_score':[f1_score]})
    print(result)

In [42]:
conf_matrix(test_labels, y_pred)

[[10881  1619]
 [ 2932  9568]]
   Accuracy  Precision   Recall  f1_score
0   0.81796   0.855278  0.76544  0.807869


In [23]:
titles_options = [("Confusion matrix, without normalization", None)]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(model, x_test, y_test,
                                 display_labels=["0","1"],
                                 cmap=plt.cm.Blues,
                                normalize=normalize,values_format='10f')
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

plt.show()

NameError: ignored

## Model 2 - Logistic Regression

In [45]:
log = LogisticRegression()
log.fit(x_train, train_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [46]:
log_pred = log.predict(x_test)
print(accuracy_score(test_labels, log_pred))

0.83148


In [47]:
conf_matrix(test_labels, log_pred)

[[10608  1892]
 [ 2321 10179]]
   Accuracy  Precision   Recall  f1_score
0   0.83148   0.843261  0.81432  0.828538


In [0]:
def logisticregression(thres):
  clf = LogisticRegression(solver = 'lbfgs')
  clf.fit(x_train, train_labels)
  pred = (clf.predict_proba(x_test)[:,1]>=thres)
  conf_mat = confusion_matrix(pred, test_labels)
  acc = accuracy_score(pred, test_labels)
  TN = conf_mat[0][0]
  FP = conf_mat[0][1]
  FN = conf_mat[1][0]
  TP = conf_mat[1][1]
  Accuracy = (TP+TN)/(TP+FP+TN+FN)
  Error = (FP+FN)/(TP+FP+TN+FN)
  Precision = TP/(TP+FP)
  Recall = TP/(TP+FN)
  return pd.Series([i, Accuracy, Error, Precision, Recall], 
                       index=["Threshold",'Accuracy', 'Error', 'Precision', 'Recall'])
    

In [0]:
log_df = pd.DataFrame(columns=["Threshold",'Accuracy', 'Error', 'Precision', 'Recall'])
log_thres = [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for i in log_thres:
    log_df = log_df.append(logisticregression(i),ignore_index=True)
display(log_df)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local

,Threshold,Accuracy,Error,Precision,Recall
0,0.2,0.83164,0.16836,0.84968,0.820091
1,0.3,0.83228,0.16772,0.83672,0.829355
2,0.4,0.83248,0.16752,0.82616,0.836736
3,0.5,0.83204,0.16796,0.81480,0.843898
4,0.6,0.83116,0.16884,0.80384,0.850300
5,0.7,0.83132,0.16868,0.79400,0.858044
6,0.8,0.82816,0.17184,0.77624,0.866185
7,0.9,0.82140,0.17860,0.74848,0.876276


## Model 3 - LDA

In [48]:
lda = LinearDiscriminantAnalysis()
lda.fit(x_train, train_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)

In [0]:
lda_pred = lda.predict(x_test)
print(accuracy_score(test_labels, lda_pred))

0.79048


In [0]:
conf_matrix(test_labels, lda_pred)

[[10045  2455]
 [ 2783  9717]]
   Accuracy  Precision   Recall  f1_score
0   0.79048   0.798308  0.77736  0.787695


## Model 4- Decision Tree

In [0]:
dt = DecisionTreeClassifier()
dt.fit(x_train, train_labels)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [0]:
dt_pred = dt.predict(x_test)
print(accuracy_score(test_labels, dt_pred))

0.7218


In [0]:
conf_matrix(test_labels, dt_pred)

[[9056 3444]
 [3511 8989]]
   Accuracy  Precision   Recall  f1_score
0    0.7218   0.722995  0.71912  0.721052


## ROC

In [38]:
nb_pred_prob = clf.predict_proba(x_test)
nb_pred_prob = nb_pred_prob[:,1]
auc = roc_auc_score(test_labels, nb_pred_prob)
fpr, tpr,thres = roc_curve(test_labels,nb_pred_prob)
plt.plot(fpr, tpr, color='brown', label='Naive Bayes ROC (AUC = %0.2f)' % auc)
plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
plt.legend(loc="lower right")
plt.title('Receiver Operating Characteristic (ROC) Curve')

logreg_pred_prob = log.predict_proba(x_test)
logreg_pred_prob = logreg_pred_prob[:,1]
auc = roc_auc_score(test_labels, logreg_pred_prob)
fpr, tpr,thres = roc_curve(test_labels,logreg_pred_prob)
plt.plot(fpr, tpr, color='orange', label='Logistic ROC (AUC = %0.2f)' % auc)
plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
plt.legend(loc="lower right")

lda_pred_prob = lda.predict_proba(x_test)
lda_pred_prob = lda_pred_prob[:,1]
auc = roc_auc_score(test_labels, lda_pred_prob)
fpr, tpr,thres = roc_curve(test_labels,lda_pred_prob)
plt.plot(fpr, tpr, color='green', label='LDA ROC (AUC = %0.2f)' % auc)
plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
plt.legend(loc="lower right")

dt_pred_prob = dt.predict_proba(x_test)
dt_pred_prob = dt_pred_prob[:,1]
auc = roc_auc_score(test_labels, dt_pred_prob)
fpr, tpr,thres = roc_curve(test_labels,dt_pred_prob)
plt.plot(fpr, tpr, color='yellow', label='Decision Tree ROC (AUC = %0.2f)' % auc)
plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
plt.legend(loc="lower right")

f_pred_prob = forest.predict_proba(x_test)
f_pred_prob = f_pred_prob[:,1]
auc = roc_auc_score(test_labels, f_pred_prob)
fpr, tpr,thres = roc_curve(test_labels,f_pred_prob)
plt.plot(fpr, tpr, color='blue', label='Random Forest ROC (AUC = %0.2f)' % auc)
plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
plt.legend(loc="lower right")

log_adaboost_pred = log_adaboost.predict_proba(x_test)
log_adaboost_pred = log_adaboost_pred[:,1]
auc = roc_auc_score(test_labels, log_adaboost_pred)
fpr, tpr,thres = roc_curve(test_labels,log_adaboost_pred)
plt.plot(fpr, tpr, color='red', label='Adaboost with Logistic Regression ROC (AUC = %0.2f)' % auc)
plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
plt.legend(loc="lower right")

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

NameError: ignored

## Model 5 - Linear SVC

In [40]:
linear_svc = LinearSVC()
linear_svc.fit(x_train, train_labels)
linear_svc_pred =linear_svc.predict(x_test)
print(accuracy_score(linear_svc_pred, test_labels))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.81836


In [41]:
conf_matrix(linear_svc_pred, test_labels)

[[10466  2507]
 [ 2034  9993]]
   Accuracy  Precision    Recall  f1_score
0   0.81836    0.79944  0.830881  0.814857


# Ensemble Learning

## Model - Random Forest

In [0]:
def Random_Forest(estimators, dataset):
    clf = RandomForestClassifier(n_estimators=estimators)
    clf.fit(x_train, train_labels)
    if dataset == "train":
        pred = clf.predict(x_train)
        conf_mat = confusion_matrix(train_labels, pred)
    else:
        pred = clf.predict(x_test)
        conf_mat = confusion_matrix(test_labels, pred)
    TN = conf_mat[0][0]
    FP = conf_mat[0][1]
    FN = conf_mat[1][0]
    TP = conf_mat[1][1]
    Accuracy = (TP+TN)/(TP+FP+TN+FN)
    Error = (FP+FN)/(TP+FP+TN+FN)
    Precision = TP/(TP+FP)
    Recall = TP/(TP+FN)
    result = pd.Series([dataset, estimators, Accuracy, Error, Precision, Recall], 
                       index=['Dataset','Estimators','Accuracy', 'Error', 'Precision', 'Recall'])
    return result

In [0]:
random_forest_df = pd.DataFrame(columns=['Dataset','Estimators','Accuracy', 'Error', 'Precision', 'Recall'])
estimators = [1,5,10,20,50,100,200]
dataset = ['train','test']
for i in estimators:
    for j in dataset:
        random_forest_df = random_forest_df.append(Random_Forest(i,j),ignore_index=True)
display(random_forest_df)

,Dataset,Estimators,Accuracy,Error,Precision,Recall
0,train,1,0.87688,0.12312,0.882076,0.87008
1,test,1,0.67144,0.32856,0.673522,0.66544
2,train,5,0.97772,0.02228,0.976919,0.97856
3,test,5,0.73624,0.26376,0.742885,0.72256
4,train,10,0.99304,0.00696,0.997498,0.98856
5,test,10,0.77296,0.22704,0.820677,0.69856
6,train,20,0.99920,0.00080,0.999280,0.99912
7,test,20,0.80708,0.19292,0.833754,0.76712
8,train,50,0.99996,0.00004,1.000000,0.99992
9,test,50,0.83724,0.16276,0.848187,0.82152


## Model - Adaboost

In [0]:
def Adaboost(estimators, dataset):
    clf = AdaBoostClassifier(n_estimators= estimators)
    clf.fit(x_train, train_labels)
    if dataset == "train":
        pred = clf.predict(x_train)
        conf_mat = confusion_matrix(pred,train_labels)
    else:
        pred = clf.predict(x_test)
        conf_mat = confusion_matrix(pred,test_labels)
    TN = conf_mat[0][0]
    FP = conf_mat[0][1]
    FN = conf_mat[1][0]
    TP = conf_mat[1][1]
    Accuracy = (TP+TN)/(TP+FP+TN+FN)
    Error = (FP+FN)/(TP+FP+TN+FN)
    Precision = TP/(TP+FP)
    Recall = TP/(TP+FN)
    result = pd.Series([dataset, estimators, Accuracy, Error, Precision, Recall], 
                       index=['Dataset','Estimators','Accuracy', 'Error', 'Precision', 'Recall'])
    return result

In [45]:
adaboost_df = pd.DataFrame(columns=['Dataset','Estimators','Accuracy', 'Error', 'Precision', 'Recall'])
estimators = [10]
dataset = ['train','test']
for i in estimators:
    for j in dataset:
        adaboost_df = adaboost_df.append(Adaboost(i,j),ignore_index=True)
display(adaboost_df)

,Dataset,Estimators,Accuracy,Error,Precision,Recall
0,train,10,0.72192,0.27808,0.86768,0.671829
1,test,10,0.72896,0.27104,0.86472,0.680068


In [33]:
log_adaboost = AdaBoostClassifier(base_estimator=LogisticRegression(), n_estimators=12)
log_adaboost.fit(x_train, train_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=LogisticRegression(C=1.0, class_weight=None,
                                                     dual=False,
                                                     fit_intercept=True,
                                                     intercept_scaling=1,
                                                     l1_ratio=None,
                                                     max_iter=100,
                                                     multi_class='warn',
                                                     n_jobs=None, penalty='l2',
                                                     random_state=None,
                                                     solver='warn', tol=0.0001,
                                                     verbose=0,
                                                     warm_start=False),
                   learning_rate=1.0, n_estimators=12, random_state=None)

# log adaboost

In [37]:
log_adaboost_pred = log_adaboost.predict(x_test)
print(accuracy_score(log_adaboost_pred, test_labels))
conf_matrix(test_labels,log_adaboost_pred)

0.8848
[[11066  1434]
 [ 1446 11054]]
   Accuracy  Precision   Recall  f1_score
0    0.8848    0.88517  0.88432  0.884745


# FF Neural network

In [0]:
x_train_ffnn = tf.keras.utils.normalize(x_train, axis=1)
x_test_ffnn = tf.keras.utils.normalize(x_test, axis=1)

In [64]:
x_train_ffnn.shape
print(type(x_train_ffnn))

<class 'numpy.ndarray'>


In [0]:
y_train = np.array(train_labels)
y_test = np.array(test_labels)

In [1]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(5000, activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(1000, activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(100, activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(2, activation=tf.nn.softmax))
model.compile(optimizer = 'adam' , 
              loss ='sparse_categorical_crossentropy', 
              metrics = ['accuracy'])
model.fit(x_train_ffnn,y_train , epochs = 3)

NameError: ignored

In [69]:
val_loss, val_acc = model.evaluate(x_test_ffnn,y_test)
print(val_loss, val_acc)

25000/25000 [==============================] - 3s 105us/sample - loss: 1.6512 - acc: 0.8549
1.6512077504180753 0.85492


# CNN with cleaned text

In [0]:
x_train_cnn = np.reshape(x_train, (25000,10000,1))
x_test_cnn = np.reshape(x_test, (25000,10000,1))

In [0]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv1D(32, 3,activation=tf.nn.relu))
model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(100,activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(2,activation=tf.nn.softmax))
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x_train_cnn, y_train,
          epochs=3,
          verbose=1)

Train on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 8s 312us/sample - loss: 4.2258 - acc: 0.4990
Epoch 2/3
25000/25000 [==============================] - 8s 316us/sample - loss: 24.0555 - acc: 0.5030
Epoch 3/3
25000/25000 [==============================] - 8s 334us/sample - loss: 51.9079 - acc: 0.4996


In [0]:
loss, acc = model.evaluate(x_test_cnn, y_test, verbose=1)
print(loss, acc)

25000/25000 [==============================] - 4s 170us/sample - loss: 50.1977 - acc: 0.5618
50.19767705078125 0.56184


# LSTM with cleaned text

In [0]:
x_train_lstm = tf.keras.utils.normalize(x_train, axis=1)
x_test_lstm = tf.keras.utils.normalize(x_test, axis=1)

In [0]:
tokenizer_lstm = tf.keras.preprocessing.text.Tokenizer(num_words= 10000)
tokenizer_lstm.fit_on_texts(x_train_lstm)
list_tokenized_train = tokenizer_lstm.texts_to_sequences(x_train_lstm)
list_tokenized_test = tokenizer_lstm.texts_to_sequences(x_test_lstm)

AttributeError: ignored

In [0]:
max_words = 500
X_train_l = sequence.pad_sequences(x_train_lstm, maxlen=max_words)
X_test_l = sequence.pad_sequences(x_test_lstm, maxlen=max_words)

In [0]:
embedding_size=32
model=tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(500, embedding_size, input_length=500))
model.add(tf.keras.layers.LSTM(100))
model.add(tf.keras.layers.Dense(2, activation='sigmoid'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           16000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 69,402
Trainable params: 69,402
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [0]:
batch_size = 64
num_epochs = 3
X_valid, y_valid = x_train_lstm[:batch_size], y_train[:batch_size]
X_train2, y_train2 = x_train_lstm[batch_size:], y_train[batch_size:]
model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), 
        batch_size= batch_size, epochs=num_epochs)

ValueError: ignored